In [1]:
# run the BM25_search to get the result
import numpy as np
import pandas as pd
import re
from collections import defaultdict
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
import nltk
import math
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# create stopwords list
stop_words = set(stopwords.words('english'))
stem = PorterStemmer()

In [26]:
data = pd.read_csv('/content/drive/MyDrive/IR_Project/anime_with_synopsis.csv')
data['Input Data'] = data['Name'].apply(str)  + " " + data['Genres'].apply(str) + " " + data['sypnopsis'].apply(str) 


data


,MAL_ID,Name,Score,Genres,sypnopsis,Input Data
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...","Cowboy Bebop Action, Adventure, Comedy, Drama,..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ...","Cowboy Bebop: Tengoku no Tobira Action, Drama,..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0...","Trigun Action, Sci-Fi, Adventure, Comedy, Dram..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...,"Witch Hunter Robin Action, Mystery, Police, Su..."
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...,"Bouken Ou Beet Adventure, Fantasy, Shounen, Su..."
...,...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...,"Daomu Biji Zhi Qinling Shen Shu Adventure, Mys..."
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...,"Mieruko-chan Comedy, Horror, Supernatural ko i..."
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .,"Higurashi no Naku Koro ni Sotsu Mystery, Demen..."
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.,"Yama no Susume: Next Summit Adventure, Slice o..."


In [4]:
input=data['Input Data'].tolist()
len(input)

16214

In [5]:
# clean and uniform the text
def preprocess_string(s):
  s = re.sub(r"won't", "will not", s)
  s = re.sub(r"can\'t", "can not", s)
  s = re.sub(r"n\’t", " not", s)
  s = re.sub(r"\’re", " are", s)
  s = re.sub(r"\’s", " is", s)
  s = re.sub(r"\’d", " would", s)
  s = re.sub(r"\’ll", " will", s)
  s = re.sub(r"\’t", " not", s)
  s = re.sub(r"\’ve", " have", s)
  s = re.sub(r"\’m", " am", s)
  s = re.sub(r'[0–9]+', '', s)
  s=re.sub(r'[^\w\s]','' '', s)
  s=stem.stem(s)
  return s

In [61]:
# def preprocess(docs):
#   new_docs=[]
#   for doc in docs:
#     new_doc=[]
#     tmp={}
#     for word in doc.lower().split():
#       if word not in stop_words:
#         a=preprocess_string(word)
#         a=stem.stem(a)
#         tmp[a]=tmp.get(a,0)+1
#         new_doc.append(a)
#     f.append(tmp)
#     new_docs.append(new_doc)
#   return new_docs


In [13]:
f=[]
tf={}
idf={}
k1=1.5
b=0.75

def init(docs):
  D=len(docs)
  for doc in docs:
    tmp={}
    for word in doc.lower().split():
      if word not in stop_words:
        a=preprocess_string(word)
        a=stem.stem(a)
        tmp[a]=tmp.get(a,0)+1
    f.append(tmp)
    for k in tmp.keys():
      tf[k]=tf.get(k,0)+1
  for k,v in tf.items():
    idf[k]=math.log(D-v+0.5) - math.log(v+0.5)

  avgd=sum(len(doc)+0.0 for doc in docs)/D
  return D,avgd

In [18]:
def sim(doc,index):
  score =0.0
  for word in doc:
    if word not in f[index]:
      continue
    else:
      d=len(f[index])
      score +=(idf[word]* f[index][word]*(k1+1)/(f[index][word]+k1*(1-b+b*d/avgd)))
  return score
def sim_overall(doc):
  scores=[]
  for index in range(D):
    score =sim(doc,index)
    scores.append(score)
  return scores

In [14]:
D,avgd=init(input)


In [15]:
D,avgd

(16214, 428.74935241149626)

In [ ]:
tf

In [ ]:
idf

In [38]:
##
import heapq
res=sim_overall('comedy')
re1=map(res.index,heapq.nlargest(5,res))
re2=heapq.nlargest(5,res)
print(list(re1),re2)

[4263, 11929, 5969, 5528, 5529] [23.399245271890756, 20.37090799954332, 18.879770132141424, 15.514535571478126, 15.457183402492502]


In [41]:
# re1 is the index of the dataframe row number for top 5 highest, re2 is the score
def BM25_search(string):
  res=sim_overall(string)
  re1=map(res.index,heapq.nlargest(5,res))
  re2=heapq.nlargest(5,res)
  return(list(re1),re2)

In [42]:
BM25_search('school')

[4263, 903, 348, 15435, 521] [40.71399743914366, 26.941557483397894, 26.611868662928092, 26.199655764695105, 24.512150626047088]


In [43]:
data.loc[4263]

MAL_ID                                                     7245
Name                  Sekai Meisaku Douwa: Wow! Maerchen Oukoku
Score                                                      6.52
Genres                                                  Fantasy
sypnopsis     Based on Western tales from the usual suspects...
Input Data    Sekai Meisaku Douwa: Wow! Maerchen Oukoku Fant...
Name: 4263, dtype: object